In [187]:
import numpy as np

def text_to_numbers(text):
    """Convert a string of uppercase letters to a list of numbers (A=0, B=1, ..., Z=25)."""
    return [ord(char) - ord('A') for char in text]

def numbers_to_text(numbers):
    """Convert a list of numbers back to uppercase text (0=A, 1=B, ..., 25=Z)."""
    return ''.join(chr(num + ord('A')) for num in numbers)

In [205]:
def find_modular_inverse(determinant, modulus):
    """
    Find the modular multiplicative inverse of `determinant` modulo `modulus`.

    Parameters:
    determinant (int): The determinant value.
    modulus (int): The modulus value.

    Returns:
    int or None: The modular inverse if it exists, otherwise None.
    """
    # Iterate through possible values for x (1 to modulus - 1)
    for x in range(1, modulus):
        if int((determinant * x) % modulus) == 1:
            return x 

    return None

In [211]:
def hill_break(ciphertext, known_plaintext):
    """
    Breaks the hill cipher on a given encrypted text by using the known plaintext attack

    Parameters:
    - ciphertext (str): Ciphertext segment to use.
    - known_plaintext (str): Corresponding known plaintext segment.

    Returns:
    - np.array: Key matrix used for encryption of the given text
    """
    known_plaintext_numbers = text_to_numbers(known_plaintext)
    ciphertext_numbers = text_to_numbers(ciphertext[:9])
    
    equations = np.zeros((len(known_plaintext_numbers), 9), dtype=int)

    for i in range(len(known_plaintext_numbers)):
        base_index = (i // 3) * 3
        if i % 3 == 0:
            equations[i, :3] = known_plaintext_numbers[base_index:base_index + 3]
        elif i % 3 == 1:
            equations[i, 3:6] = known_plaintext_numbers[base_index:base_index + 3]
        elif i % 3 == 2:
            equations[i, 6:] = known_plaintext_numbers[base_index:base_index + 3]
            
    inverse_equations = np.linalg.inv(equations)
    equation_determinant = np.linalg.det(equations)
    multiplied =  equation_determinant * inverse_equations
    
    modular_inverse_value = find_modular_inverse(equation_determinant, 26)
    final = np.mod(np.mod(multiplied, 26) * modular_inverse_value, 26)

    return np.mod(final.dot(np.array(ciphertext_numbers).reshape(-1, 1)), 26).round().reshape(3, 3)


In [212]:
def mod_inverse(a, m):
    """Return the modular inverse of a under modulo m."""
    m0, x0, x1 = m, 0, 1
    if m == 1:
        return 0
    while a > 1:
        q = a // m
        m, a = a % m, m
        x0, x1 = x1 - q * x0, x0
    if x1 < 0:
        x1 += m0
    return x1

def get_inverse_key_matrix(A):
    def determinant_mod_26(matrix):
        """
         # Function to compute the determinant mod 26
        :param matrix: 
        :return: 
        """
        det = int(round(np.linalg.det(matrix)))  # Calculate determinant
        return det % 26
    
    def mod_inverse(a, mod=26):
        """
        # Function to find modular multiplicative inverse of a number with a given modulo
        :param a: 
        :param mod: 
        :return: 
        """
        for x in range(1, mod):
            if (a * x) % mod == 1:
                return x
        return None
    
    def adjugate_mod_26(matrix):
        """
        Function to compute the adjugate matrix mod 26
        :param matrix: 
        :return: 
        """
        cofactor_matrix = np.zeros((3, 3), dtype=int)
        
        # Compute cofactor matrix
        for i in range(3):
            for j in range(3):
                minor = np.delete(np.delete(matrix, i, axis=0), j, axis=1)
                cofactor = ((-1) ** (i + j)) * int(round(np.linalg.det(minor)))
                cofactor_matrix[i, j] = cofactor
        
        # Adjugate is the transpose of the cofactor matrix
        adjugate = cofactor_matrix.T % 26
        return adjugate
    
    # Calculate determinant and its mod 26 inverse
    det_A = determinant_mod_26(A)
    det_A_inv = mod_inverse(det_A)
    
    # Compute adjugate of A mod 26
    adj_A = adjugate_mod_26(A)
    
    # Multiply the adjugate by the determinant inverse, then take mod 26 for each entry
    A_inv_mod_26 = (det_A_inv * adj_A) % 26
    
    # Display the inverse matrix
    return A_inv_mod_26


def hill_cipher_decode(ciphertext, key_matrix):
    """Decrypts a ciphertext using the Hill cipher with the given key matrix."""
    # Prepare ciphertext as numerical vectors
    ciphertext_vector = [ord(char) - ord('A') for char in ciphertext]
    
    # Reshape ciphertext into blocks matching key matrix size
    n = key_matrix.shape[0]
    ciphertext_blocks = [ciphertext_vector[i:i+n] for i in range(0, len(ciphertext_vector), n)]
    
    # Get inverse key matrix
    inv_key_matrix = get_inverse_key_matrix(key_matrix)
        
    # Decrypt each block
    decrypted_text = ""
    for block in ciphertext_blocks:
        decrypted_block = np.dot(inv_key_matrix, block) % 26
        decrypted_text += ''.join(chr(num + ord('A')) for num in decrypted_block)
    
    return decrypted_text

In [213]:
ciphertext = 'BALQTGFGYNFUHVLOIVCGPRZJUTHGWOVWCWAJGWN'
known_plaintext = 'DRAHYJURA'

key_matrix = hill_break(ciphertext, known_plaintext)
hill_cipher_decode(ciphertext, key_matrix)

'DRAHYJURAJPRIDDNESVECERZAMNOUMILUJEMTAA'

In [214]:
ciphertext = 'PCPOVZOJYEJXJLVINLJMIAVAVEUKZLERO'
known_plaintext = 'DRAHYJURA'

key_matrix = hill_break(ciphertext, known_plaintext)
hill_cipher_decode(ciphertext, key_matrix)

'DRAHYJURAJUZZAMNONECHODMAMDRUHEHO'

In [215]:
ciphertext = 'NMUSMRFJGRWSWVKKDJKYTYTNSVMOJW'
known_plaintext = 'DRAHYJURA'

key_matrix = hill_break(ciphertext, known_plaintext)
hill_cipher_decode(ciphertext, key_matrix)

'DRAHYJURAJBOLASOMHLUPAODPUSTMI'